In [2]:
import cv2
cap =cv2.VideoCapture(0) #0 is index of camera
while True : #infinite loop for video capture
    ret,frame= cap.read()
    if not ret:
        print("Error: Failed to capture frame from camera")
        break
    cv2.imshow("Webcam", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
cap.release()
cv2.destroyAllWindows()

In [12]:
def play_video(video_path):
    cap =cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: could not open video file")
        return
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        cv2.imshow("Video",frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break 
    cap.release()
    cv2.destroyAllWindows()
video_path = (r"C:\Users\AMZ PC\OneDrive\Desktop\CV\Smoke detection\3826855-hd_1920_1080_24fps.mp4")
play_video(video_path)


In [13]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical

training_data=[r"C:\Users\AMZ PC\OneDrive\Desktop\CV\Smoke detection\fire_dataset\fire_images", 
               r"C:\Users\AMZ PC\OneDrive\Desktop\CV\Smoke detection\fire_dataset\non_fire_images"]

def load_images(training_data):
    images = []
    labels = []
    for i in range(len(training_data)):
        folder = training_data[i]
        label = i  # 0 for fire, 1 for non-fire
        for filename in os.listdir(folder):
            try:
                img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    # Resize image to 48x48 pixels
                    img = cv2.resize(img, (48, 48))
                    images.append(img)
                    labels.append(label)  # Correct label assignment
                else:
                    print(f"Warning: Failed to load {filename}")
            except Exception as e:
                print(f"Error: loading image {os.path.join(folder, filename)}: {e}")
    
    # Convert lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    # Debugging print statements to check data
    print(f"Total images loaded: {len(images)}")
    print(f"Images shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"Sample labels: {labels[:10]}")  # Print first 10 labels for debugging

    return images, labels

# Load images and labels
images, labels = load_images(training_data)

# Further Debugging
print(f"Type of images array: {type(images)}")
print(f"Type of labels array: {type(labels)}")
print(f"First image shape: {images[0].shape}")  # Check shape of first image


Total images loaded: 998
Images shape: (998, 48, 48)
Labels shape: (998,)
Sample labels: [0 0 0 0 0 0 0 0 0 0]
Type of images array: <class 'numpy.ndarray'>
Type of labels array: <class 'numpy.ndarray'>
First image shape: (48, 48)


In [14]:
#preprocessing
x_train, x_test , y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train= x_train.reshape(x_train.shape[0],48,48,1).astype('float32')/255
x_test= x_test.reshape(x_test.shape[0],48,48,1).astype('float32')/255

y_train = to_categorical(y_train) 
y_test = to_categorical(y_test)

model=Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [15]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=1, validation_data=(x_test, y_test))
model.save("fire_detection.model.h5")

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 269ms/step - accuracy: 0.6113 - loss: 0.7700 - val_accuracy: 0.7300 - val_loss: 0.5271
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 236ms/step - accuracy: 0.7637 - loss: 0.5339 - val_accuracy: 0.8250 - val_loss: 0.3916
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 241ms/step - accuracy: 0.8245 - loss: 0.3848 - val_accuracy: 0.8250 - val_loss: 0.3467
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 231ms/step - accuracy: 0.8343 - loss: 0.3695 - val_accuracy: 0.8450 - val_loss: 0.3164
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 239ms/step - accuracy: 0.8775 - loss: 0.2790 - val_accuracy: 0.8750 - val_loss: 0.3005
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 238ms/step - accuracy: 0.8671 - loss: 0.3040 - val_accuracy: 0.8850 - val_loss: 0.3062
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 235ms/step - accuracy: 0.8841 - loss: 0.2694 - val_accuracy: 0.8650 - val_loss: 0.3109
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 236ms/step - accuracy: 0.8845 - loss: 0.2658 - val_accuracy: 0.

In [25]:
import cv2
from keras.models import load_model
import numpy as np
import os

# Load the pre-trained fire detection model
model = load_model("fire_detection.model.h5")

def detect_fire(frame, threshold=0.5):
    # Resize and preprocess the frame (assuming model expects grayscale 48x48 input)
    preprocess_frame = cv2.cvtColor(cv2.resize(frame, (48, 48)), cv2.COLOR_BGR2GRAY)
    preprocess_frame = np.expand_dims(preprocess_frame, axis=0)
    preprocess_frame = np.expand_dims(preprocess_frame, axis=-1)
    preprocess_frame = preprocess_frame.astype("float32") / 255.0

    # Make prediction using the model
    prediction = model.predict(preprocess_frame)

    # Check if fire is detected based on the model's output and the threshold
    if prediction[0][1] >= threshold:
        return True
    else:
        return False

# Open the video file
cap = cv2.VideoCapture(r"C:\Users\AMZ PC\OneDrive\Desktop\CV\Smoke detection\3826855-hd_1920_1080_24fps.mp4")
if not cap.isOpened():
    print("Error: could not open video file")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Check if fire is detected in the frame
    if detect_fire(frame):
        # Draw a rectangle and display a warning message
        cv2.rectangle(frame, (100, 100), (frame.shape[1] - 100, frame.shape[0] - 100), (0, 0, 255), 2)
        cv2.putText(frame, "Warning, fire detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Display the processed video
    cv2.imshow("Video", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
